 Cursor control
 Left click (fist)
 Volume control (thumb-index gap)
 Scrolling (index + middle swipe)
 Brightness control (ring + thumb gap)
 App switching (4 fingers up)
 On-screen keyboard that opens for typing with click



In [1]:
pip install opencv-python cvzone autopy numpy pyautogui screen-brightness-control --upgrade

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import numpy as np
import autopy
import pyautogui
import platform
import subprocess
import time
import screen_brightness_control as sbc
from cvzone.HandTrackingModule import HandDetector

In [3]:
is_windows = platform.system() == "Windows"
if is_windows:
    from ctypes import cast, POINTER
    from comtypes import CLSCTX_ALL
    from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

    devices = AudioUtilities.GetSpeakers()
    interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
    volume = cast(interface, POINTER(IAudioEndpointVolume))
    volRange = volume.GetVolumeRange()
    minVol, maxVol = volRange[0], volRange[1]
else:
    def set_volume_mac(vol_percent):
        vol_percent = max(0, min(100, vol_percent))
        subprocess.run(["osascript", "-e", f"set volume output volume {int(vol_percent)}"])


In [4]:
# Initialize webcam
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

True

In [ ]:
# Initialize hand detector
detector = HandDetector(detectionCon=0.8, maxHands=1)

screen_w, screen_h = autopy.screen.size()

I0000 00:00:1753273951.791862 45568122 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M1


In [6]:
# State flags
click_down = False
scrolling = False
keyboard_opened = False

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [7]:
# Smoothening
plocX, plocY = 0, 0
clocX, clocY = 0, 0
smoothening = 7

W0000 00:00:1753273951.837347 45568311 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


W0000 00:00:1753273951.917789 45568311 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [8]:
def open_keyboard():
    global keyboard_opened
    if not keyboard_opened:
        if is_windows:
            subprocess.Popen('osk') 
        else:
            subprocess.Popen(['open', '-a', 'KeyboardViewer']) 
        keyboard_opened = True

In [9]:
def set_brightness_safe(value):
    try:
        sbc.set_brightness(int(value))
    except Exception as e:
        print(f"Brightness control error: {e}")


In [ ]:
while True:
    success, img = cap.read()
    img = cv2.flip(img, 1)
    hands, img = detector.findHands(img, flipType=False)
    if hands:
        hand = hands[0]
        lmList = hand["lmList"]
        fingers = detector.fingersUp(hand)
        x1, y1 = lmList[8][0], lmList[8][1]
        screen_x = np.interp(x1, [100, 540], [0, screen_w])
        screen_y = np.interp(y1, [100, 380], [0, screen_h])
        clocX = plocX + (screen_x - plocX) / smoothening
        clocY = plocY + (screen_y - plocY) / smoothening
        clocX = max(0, min(screen_w - 1, clocX))
        clocY = max(0, min(screen_h - 1, clocY))
        
        if fingers[1] == 1 and fingers[2] == 0:
            autopy.mouse.move(clocX, clocY)
            plocX, plocY = clocX, clocY
        if fingers == [0,0,0,0,0]:
            if not click_down:
                autopy.mouse.click()
                click_down = True
                cv2.putText(img, "CLICK", (x1+20, y1), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 3)
        else:
            click_down = False
        length, info, img = detector.findDistance(lmList[4][0:2], lmList[8][0:2], img)
        volPer = np.interp(length, [30, 200], [0, 100])

        if is_windows:
            vol = np.interp(length, [30, 200], [minVol, maxVol])
            volume.SetMasterVolumeLevel(vol, None)
        else:
            set_volume_mac(volPer)
        length_bright, info, img = detector.findDistance(lmList[4][0:2], lmList[16][0:2], img)
        bright = np.interp(length_bright, [30, 200], [0, 100])
        set_brightness_safe(bright)
        if fingers[1] == 1 and fingers[2] == 1:
            if not scrolling:
                pyautogui.scroll(-50) 
                scrolling = True
        else:
            scrolling = False
        if fingers == [0,1,1,1,1]:
            if is_windows:
                pyautogui.keyDown('alt')
                pyautogui.press('tab')
                pyautogui.keyUp('alt')
            else:
                pyautogui.keyDown('command')
                pyautogui.press('tab')
                pyautogui.keyUp('command')
            time.sleep(1)
        if fingers == [0,0,0,0,1]:
            open_keyboard()
        cv2.putText(img, f'Volume: {int(volPer)}%', (10, 70),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
        cv2.putText(img, f'Brightness: {int(bright)}%', (10, 100),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
    
    cv2.putText(img, "PC Gesture Control | 'q' to quit", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)
    
    cv2.imshow("PC Control", img)
    
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

2025-07-23 18:02:32.652 Python[5844:45568122] WARNING: Secure coding is automatically enabled for restorable state! However, not on all supported macOS versions of this application. Opt-in to secure coding explicitly by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState:.
/Users/ankitkumar/Library/Python/3.9/lib/python/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Brightness control error: name '_OS_MODULE' is not defined


/Users/ankitkumar/Library/Python/3.9/lib/python/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Brightness control error: name '_OS_MODULE' is not defined


/Users/ankitkumar/Library/Python/3.9/lib/python/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


...

Brightness control error: name '_OS_MODULE' is not defined


/Users/ankitkumar/Library/Python/3.9/lib/python/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Brightness control error: name '_OS_MODULE' is not defined


KeyboardInterrupt: 

: 